<a href="https://colab.research.google.com/github/ghazalrafiei/Resume-Recommender-System/blob/main/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !unzip multi_cased_L-12_H-768_A-12.zip

In [ ]:
pip install transformers

     |████████████████████████████████| 2.9 MB 5.1 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
     |████████████████████████████████| 895 kB 74.8 MB/s 
     |████████████████████████████████| 3.3 MB 48.2 MB/s 
     |████████████████████████████████| 596 kB 59.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cpu
CUDA is not available.  Training on CPU ...


In [ ]:
# model.

In [ ]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz

In [ ]:
!pip install hazm
# !pip install parsivar

     |████████████████████████████████| 316 kB 5.2 MB/s 
     |████████████████████████████████| 233 kB 51.1 MB/s 
     |████████████████████████████████| 1.4 MB 50.0 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=088f31a947529da2cc10ce8ca8a9c885c590235ee3346cf932b945ae0c693210
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154721 sha256=95209e8c8c6a9a3f3a10b0a9e1b0d59957d16c95420f0f5e5f716bead5e78843
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
# !gunzip /content/cc.fa.300.bin.gz

In [ ]:
import hazm
import pandas as pd

In [ ]:
stop_words = set()
def loads_stops():
    h = hazm.Normalizer()
    with open('/content/drive/MyDrive/resume/en_stopwords.txt','r+') as s:
        for w in s.readlines():
            w = w.rstrip().replace('\u200c',' ')
            w = h.normalize(w)
            stop_words.add(w)
loads_stops()

In [ ]:
syn_df =  pd.read_csv('/content/drive/MyDrive/resume/synonyms.csv')
eng_to_fa = {}
for i in range(len(syn_df)):
  eng_to_fa[syn_df['eng'][i]] = syn_df['fa'][i]

In [ ]:
!pip install parsivar
from parsivar import Normalizer,Tokenizer 
_normalizer = Normalizer()

     |████████████████████████████████| 36.2 MB 68 kB/s 
     |████████████████████████████████| 1.5 MB 38.4 MB/s 
  Created wheel for parsivar: filename=parsivar-0.2.3-py3-none-any.whl size=36492971 sha256=9d5788560526c04ea7d21be28636dcbfc1c4d69b9a503516bc846a8196398d49
  Stored in directory: /root/.cache/pip/wheels/ae/67/7a/49cbf08f64d3f76a26eceaf0e481a40e233f05d4356875cbed
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449921 sha256=5f49743960dbad48fec46c29e6e7e486c21461c30297d6fb5a366e6b90e3cc39
  Stored in directory: /root/.cache/pip/wheels/48/8b/7f/473521e0c731c6566d631b281f323842bbda9bd819eb9a3ead
Successfully built parsivar nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.3
    Uninstalling nltk-3.3:
      Successfully uninstalled nltk-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
hazm 0.7.0 requires 

In [ ]:
def normaz(t):
  _normalizer = Normalizer()
  return _normalizer.normalize(t)

In [ ]:
def tokenizePersianText(text):
  x = normaz(text)
  x = x.replace('\u200c',' ')
  x = x.replace('\u202a',' ')
  x = x.replace('\u202c',' ')
  x = x.replace('\u202b',' ')
  x = x.replace('\u202c',' ')

  # ^((\\u202a)|(\\u202c)|(\\x0c)|(\\uf0b7)|\\ue812|\\u200c)
  tz = Tokenizer()
  tokens = tz.tokenize_words(x)
  # s = SpellCheck()
  # tokens = [s.spell_corrector(w) for w in text]

  tokens = [w for w in tokens if not w in stop_words if len(w)>3 and w!='C']
  for t in tokens:
    if re.match(r'[a-zA-Z]',t):
      if t.lower() in eng_to_fa.keys():
        tokens.remove(t)
        tokens.append(eng_to_fa[t.lower()])
  return tokens

In [ ]:
import re
description = ''
with open('/content/drive/MyDrive/resume/desc4.txt') as f:
  for r in f.readlines():
    description += ' '+(r.replace('/',' ').rstrip())
desc_tokens = tokenizePersianText(description)

In [ ]:
import re
resumes = {}
emails = {}
numbers = {}
phones = {}
urls = {}
educations = {}
university_experience = {}
enterprise_experience = {}
  
university_experience = {}
for i in range(1,198):
  resume = ''
  emails[i] = []
  phones[i] = []
  urls[i] = []
  numbers[i] = []
  educations[i] = []
  university_experience[i] = []
  enterprise_experience[i] = []
  
  with open(f'/content/drive/MyDrive/resume/eng_pdfs/resume{i}.txt') as f:
    for lines in f.readlines():
      line = lines.replace('\n',' ')
      emails[i]+=re.findall(r'[0-9a-zA-Z\.-]+@[a-zA-Z\.-]+',line)
      line = re.sub(r'[0-9a-zA-Z\.-]+@[a-zA-Z\.-]+',' ',line)

      # phones[i]+=re.findall(r'[\+98|0]?9\d{9}',line)
      # re.sub(r'[\+98|0]?9\d{9}',' ',line)

      # phones[i]+=re.findall(r'\+\u06F9\u06F8|\u06F90]?\u06F9{\u06F0-\u06F9}{\u06F9}',line)
      # re.sub(r'\+\u06F9\u06F8|\u06F90]?\u06F9{\u06F0-\u06F9}{\u06F9}',' ',line)

      numbers[i]+=re.findall(r'(\d+|[\u06F0-\u06F9]+)',line)
      line = re.sub(r'(\d+|[\u06F0-\u06F9]+)',' ',line)

      urls[i]+=re.findall(r'([https?:\/\/]?[www\.]?[a-z0-9A-Z\u06F0-\u06F9]+\.[a-z]+[\/[a-z0-9]]?)',line)
      line = re.sub(r'([https?:\/\/]?[www\.]?[a-z0-9A-Z\u06F0-\u06F9]+\.[a-z]+[\/[a-z0-9]]?)',' ',line)
      
      linen = _normalizer.normalize(line)
      educations[i]+=re.findall(r'(لیسانس|کارشناسی ارشد|کارشناسی|فوق لیسانس|دکترا|دیپلم|bachelor|diploma|master|bs|b\.s|m\.s|phd)',linen)
      linen = re.sub(r'(لیسانس|کارشناسی ارشد|کارشناسی|فوق لیسانس|دکترا|دیپلم|bachelor|diploma|master|bs|b\.s|m\.s|phd)',' ',linen)
        # print(linen)

      if line.find('دانشگاه') != -1 or line.lower().find('university') != -1:
        # print(line)
        university_experience[i].append(linen)

      if line.find('شرکت')!= -1 or line.find('کارخانه')!=-1:
        # print(line)
        enterprise_experience[i].append(_normalizer.normalize(line.replace('\n',' ')))

      resume += ' '+line.rstrip()
  
  res_tokens = tokenizePersianText(resume)
  resumes[i] = res_tokens
  # resumes[i] = resume

metadata = pd.DataFrame(index=range(1,198))
metadata['email'] = emails.values()
metadata['phone'] = phones.values()
metadata['education'] = educations.values()
metadata['university'] = university_experience.values()
metadata['url'] = urls.values()
metadata['enterprise'] = enterprise_experience.values()
metadata['numbers'] = numbers.values()
# metadata


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
# metadata

In [ ]:
# general config
import os
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 3
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/content/bert-fa-base-uncased/pytorch_model.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [ ]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH)

# print(config.to_json_string())

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440 [00:00<?, ?B/s]

In [ ]:
def get_vector(sen_tokens):
  # data = tokenizer.prepare_for_tokenization(sen_tokens)
  # t = tokenizer.tokenize(data)

  # token_ids = tokenizer.convert_tokens_to_ids(sen_tokens)
  encoding = tokenizer.encode_plus(
    sen_tokens,
    max_length=320,
    truncation=True,
    add_special_tokens=False, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=None,
    return_attention_mask=True,
    padding='max_length',
    return_tensors='pt',  # Return PyTorch tensors
)

# print(f'Keys: {encoding.keys()}\n')
# for k in encoding.keys():
#     print(f'{k}:\n{encoding[k]}')
  return encoding['input_ids']

In [ ]:
# type(desc_tokens)

In [ ]:
desc_model = get_vector(desc_tokens)
resumes_model = {}
for i in range(1,35):
  resumes_model[i] = get_vector(resumes[i])

In [ ]:
# resumes_model[i]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance
import numpy as np

similarities = {}
similarities_0 = {}
similarities_1 = {}

for i in range(1,35):
  similarities_0[i] = cosine_similarity(np.array(resumes_model[i]).reshape(1,-1),np.array(desc_model).reshape(1,-1))
  similarities[i] = distance.euclidean(np.array(resumes_model[i]).reshape(1,-1),np.array(desc_model).reshape(1,-1))


In [ ]:
ranking = sorted(similarities.items(), key=lambda x: x[1], reverse=False)
result = pd.DataFrame()
result['number'] = [r[0] for r in ranking]
result['score_distance'] = [r[1] for r in ranking]
result

,number,score_distance
0,24,189221.964547
1,25,202676.573854
2,17,244244.099521
3,27,250001.729770
4,22,257047.295434
5,11,285814.011665
6,19,287499.425629
7,9,299360.497633
8,32,299889.991909
9,12,300754.202952


In [ ]:
ranking = sorted(similarities_0.items(), key=lambda x: x[1], reverse=True)
result = pd.DataFrame()
result['number'] = [r[0] for r in ranking]
result['score_cosine'] = [r[1].flatten()[0] for r in ranking]
result

,number,score_cosine
0,24,0.342160
1,3,0.280285
2,9,0.272047
3,11,0.254610
4,15,0.236405
5,28,0.235414
6,20,0.212508
7,12,0.208145
8,26,0.193155
9,13,0.174926
